<a href="https://colab.research.google.com/github/trandlong/MarketingResearch/blob/master/Twitter_connection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install tweepy --upgrade

     |████████████████████████████████| 77 kB 3.3 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [79]:
import tweepy as tw
import numpy as np
import pandas as pd
import requests
import os
import json
from datetime import date, datetime, timedelta, timezone

In [102]:

# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = my_bearer_token


# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields


def create_url(keyword,  max_results = 50):
    """ 
    create search URL
    """

    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,

                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id,attachments.media_keys',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'media.fields':'duration_ms,height,media_key,preview_image_url,public_metrics,type,url,width,alt_text',
                    'next_token': {}}
    return (search_url, query_params)

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FullArchiveSearchPython"
    return r


def connect_to_endpoint(url, params):

    """
    connect end point to URL
    """
    response = requests.request("GET", search_url, auth=bearer_oauth, params=params)

    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def search_tweet(keyword, max_results = 50):

    search_url, query_params = create_url(keyword, max_results)
    json_response = connect_to_endpoint(search_url, query_params)
    return json_response
    #(json.dumps(json_response, indent=4, sort_keys=True))




In [103]:
query_pepsi= '#pepsi has:images'

pepsi_tweets  = search_tweet(query_pepsi)


In [256]:
#populate data to data frame
tweets_df = pd.DataFrame(columns=['media_key','media_type','media_url', 'width','height', 'author_id','conversation_id','media_keys','lang','reply_settings','source','text','like_count',
                                    'user_created_at',
                                    'followers_count',
                                    'following_count',
                                    'listed_count',
                                    'tweet_count',
                                    ])


#first get media, and URL, and then get all the data from author
for a in range(len(pepsi_tweets['includes']["media"])): #iterate over the media
  try:
    media_key = pepsi_tweets['includes']["media"][a]['media_key']
    media_type = pepsi_tweets['includes']["media"][a]['type']
    media_url = pepsi_tweets['includes']["media"][a]['url']
    width = pepsi_tweets['includes']["media"][a]['width']
    height = pepsi_tweets['includes']["media"][a]['height']


    for i in range(len(pepsi_tweets["data"])):  #iterate over the tweet
      
        if "attachments" in pepsi_tweets["data"][i].keys(): #check if attachment in the tweet
          if media_key in pepsi_tweets["data"][i]["attachments"]['media_keys']:
            media_keys = pepsi_tweets["data"][i]["attachments"]['media_keys'] #grab media key of the attachment
            author_id = pepsi_tweets["data"][i]["author_id"]
            conversation_id = pepsi_tweets["data"][i]["conversation_id"]
            lang = pepsi_tweets["data"][i]["lang"]
            reply_settings = pepsi_tweets["data"][i]["reply_settings"]
            source = pepsi_tweets["data"][i]["source"]
            tweet_text = pepsi_tweets["data"][i]["text"]
            like_count =pepsi_tweets["data"][i]["public_metrics"]['like_count']
            reply_count =pepsi_tweets["data"][i]["public_metrics"]['reply_count']
            retweet_count =pepsi_tweets["data"][i]["public_metrics"]['retweet_count']
            for j in range(len(pepsi_tweets["includes"]['users'])): #find out where the user is
              if author_id == pepsi_tweets["includes"]['users'][j]['id']: #grab id from user
                user_created_at = pepsi_tweets["includes"]['users'][j]['created_at']
                followers_count = pepsi_tweets["includes"]['users'][j]['public_metrics']['followers_count']
                following_count = pepsi_tweets["includes"]['users'][j]['public_metrics']['following_count']
                listed_count = pepsi_tweets["includes"]['users'][j]['public_metrics']['listed_count']
                tweet_count = pepsi_tweets["includes"]['users'][j]['public_metrics']['tweet_count']
                
                #append data
                tweets_df = tweets_df.append(pd.DataFrame({'media_key':media_key,
                                                          'media_type':media_type,
                                                          'media_url':media_url, 
                                                          'width':width,
                                                          'height':height, 
                                                          'author_id':author_id,
                                                          'conversation_id':conversation_id,
                                                          'media_keys':media_keys,
                                                          'lang':lang,
                                                          'reply_settings':reply_settings,
                                                          'source':source,
                                                          'text':tweet_text,
                                                          'like_count':like_count,
                                                          'user_created_at':user_created_at,
                                                          'followers_count':followers_count,
                                                          'following_count':following_count,
                                                          'listed_count':listed_count,
                                                          'tweet_count':tweet_count,}))
              tweets_df = tweets_df.reset_index(drop=True)
  except:
    pass


In [257]:
tweets_df

,media_key,media_type,media_url,width,height,author_id,conversation_id,media_keys,lang,reply_settings,source,text,like_count,user_created_at,followers_count,following_count,listed_count,tweet_count
0,3_1507486051941691393,photo,https://pbs.twimg.com/media/FOuqk8XXMAEEmXL.jpg,1536,2048,1495797795814576129,1507486060091224065,3_1507486051941691393,en,everyone,Twitter Web App,Yes….YES!!!! #Pepsi finally it’s here in Ontar...,1,2022-02-21T16:29:29.000Z,11,83,0,869
1,3_1507476022308089857,photo,https://pbs.twimg.com/media/FOuhdJEXsAEvI54.jpg,2048,2048,1499483743098134529,1507476025529311235,3_1507476022308089857,en,everyone,Twitter for iPhone,#Pepsi and #IHOP Team Up for Limited-Edition P...,0,2022-03-03T20:36:13.000Z,11,37,0,48
2,3_1507474655900221440,photo,https://pbs.twimg.com/media/FOugNmzWQAATzgM.jpg,1080,1080,1493282965744496647,1507474675575959552,3_1507474655900221440,en,everyone,Twitter for Android,@elissakh @Pink @Beyonce @britneyspears throug...,0,2022-02-14T17:56:37.000Z,4,21,0,67
3,3_1507473804578021380,photo,https://pbs.twimg.com/media/FOufcDYUYAQ6nrA.jpg,2515,1553,863542868,1507473891668594691,3_1507473804578021380,und,everyone,Twitter Web App,Ooof #stevescomics #koala #Pepsi #comics https...,1,2012-10-05T19:17:14.000Z,239,392,4,518
4,3_1507468492953755655,photo,https://pbs.twimg.com/media/FOuam4CaIAcqrlQ.jpg,1536,2048,4787855712,1507468504026746883,3_1507468492953755655,ja,everyone,Twitter for Android,断捨離戦果w\nカップヌードル40周年ガンプラ付きヌードル＆pepsiガンダム缶www\n…...,21,2016-01-20T09:03:26.000Z,4099,4903,14,20975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,3_1507303234393841664,photo,https://pbs.twimg.com/media/FOsETj7XoAAK_Ap.jpg,715,1359,1488166234936029186,1507303243545825282,3_1507303234393841664,en,everyone,Twitter for Android,https://t.co/q9UrLijP2l\nIt is a link to a Cro...,0,2022-01-31T15:04:33.000Z,75,302,0,473
85,3_1507302415325962243,photo,https://pbs.twimg.com/media/FOsDj4qXoAMXPEY.jpg,718,1231,1488166234936029186,1507302438734282752,3_1507302415325962243,en,everyone,Twitter for Android,https://t.co/SHqrstcNFv\nIt is a link to a Cro...,1,2022-01-31T15:04:33.000Z,75,302,0,473
86,3_1507302415325962243,photo,https://pbs.twimg.com/media/FOsDj4qXoAMXPEY.jpg,718,1231,1488166234936029186,1507302438734282752,3_1507302426545643531,en,everyone,Twitter for Android,https://t.co/SHqrstcNFv\nIt is a link to a Cro...,1,2022-01-31T15:04:33.000Z,75,302,0,473
87,3_1507302426545643531,photo,https://pbs.twimg.com/media/FOsDkidWYAsTRHT.jpg,718,1345,1488166234936029186,1507302438734282752,3_1507302415325962243,en,everyone,Twitter for Android,https://t.co/SHqrstcNFv\nIt is a link to a Cro...,1,2022-01-31T15:04:33.000Z,75,302,0,473


In [262]:
# save
#tweets_df.to_csv("test_50tweets_data.csv")
#!cp test_50tweets_data.csv "drive/My Drive/"